# RAG 관련 테스트 프로그램 

## 1. 기본 프로그램 선언

### 1.1 관련 패키지 설치

In [ ]:
! pip install  --upgrade langchain-aws langchain-community boto3 > /dev/null 2>&1
! pip list | egrep 'boto3|langchain' 

### 1.2 프로그램 진행관련 초기 선언

In [ ]:

import boto3
import time
from langchain_aws import BedrockEmbeddings  # Embedding 을 위한
from langchain_aws import BedrockLLM  # LLM을 위한
from langchain_aws import ChatBedrock
from langchain_community.vectorstores import FAISS  # 인메모리용 백터 저장소
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
# from langchain_community.vectorstores import OpenSearchVectorSearch

# from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader

      
# AWS 자격 증명 확인
try:
    session = boto3.Session()
    credentials = session.get_credentials()
    print("AWS 자격 증명 확인 완료")
    print(f"리전: {session.region_name}")
except Exception as e:
    print(f"AWS 자격 증명 오류: {e}")
    print("AWS CLI를 사용하여 자격 증명을 설정해주세요: aws configure")

# Bedrock 클라이언트 생성 
bedrock = session.client('bedrock-runtime')
bedrock2 = session.client('bedrock')

# 현재 리전 확인
current_region = boto3.Session().region_name

# 현재 리전에 맞는 Model ID 자동 검색
models = bedrock2.list_foundation_models()['modelSummaries']

base_model=""
titan_model=""

# 버지니아 리전인 경우 Cross-region inference 사용
if current_region == 'us-east-1':
    base_model = "us.anthropic.claude-3-5-haiku-20241022-v1:0"
else:
    find_models = next((m for m in models if 'claude-3-5-haiku' in m['modelId'].lower()), None)
    base_model = find_models['modelId']

# 버지니아 리전인 경우 Cross-region inference 사용
if current_region == 'us-east-1':
    titan_model = "amazon.titan-embed-text-v2:0"
else:
    find_models = next((m for m in models if 'titan-embed-text-v2' in m['modelId'].lower()), None)
    titan_model = find_models['modelId']

titan_model = "amazon.titan-embed-text-v2:0"
print("[Model ID] claude-3-5-haiku :", base_model)
print("[Model ID] titan-embed-text:", titan_model)
print("[Current Region]:", current_region)


## 2. 문서 로드 
- 인메모리 벡터 데이터 베이스에 저장한 데이터를 읽어옴 

In [ ]:
loader = CSVLoader("ko.Amazon_SageMaker_FAQs.csv" )  # CSV 형식은 ROW 단위로 Document 를 읽어 드림.
documents = loader.load()
print(f"documents:loaded:size={len(documents)}") 

## 3. 문서 분할
- 인메모리 벡터 데이터베이스에 저장할 문서를 Chunk 단위로 나눔, 

In [ ]:
text_splitter = CharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=400,
        separator=",",  # chunk size 에 따라 검색되는 문장이 달라짐.
    )
docs = text_splitter.split_documents(documents)
print(f"Documents: split이후 and chunking size={len(docs)}")

## 3. Bedrock titan 모델을 임베딩 모델로 선언

In [ ]:
embeddings = BedrockEmbeddings( model_id=titan_model )

## 4.Test 문자를 임베딩 해보기 
- text 변수에 선언된 임시 문장을 벡터로 변환해 봅니다. 

In [ ]:
text = "This is a sample text for embedding."
text_splitter = CharacterTextSplitter(
    chunk_size=100, chunk_overlap=40, separator="\n"
)

chunks = text_splitter.split_text(text)
embedded_text = embeddings.embed_query(chunks[0])
print(chunks[0])
print(f"Embedding dimension:  {len(embedded_text)}")   # 백터에 차원 표시
print(f"Embedding vector: {embedded_text[:10]}...")    # 베터값중에 10개 까지만 출력

## 5. 백터 저장소(FAISS) 에 백터 데이터와 임베딩 값을 함께 저장한다.
- 현재 시스템의 일부 메모리 장소를 백터 데이터 저장소로 사용하는 FAISS 기능을 사용

In [ ]:
vectorstore_faiss_aws = None
try:

    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding=embeddings,
    )

except ValueError as error:
    if "AccessDeniedException" in str(error):
        print(
            f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
        \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
        \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n"
        )

        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution
    else:
        raise error

## 6. 벡터를 이용하여 질의해 본다.

### 6.1 질문 정의
- 백터를 검색할 때 사용한 질문을 선언한다. 
- 현재 벡터에 있는 데이터는 Smazon Sagemaker관련한 CSV 파일이 저장되어 있어서 관련된 'Sagemaker" 질문을 한다.

In [ ]:
query = "세이지메이커 모델카드는 무엇인가?"

### 6.2 벡터 저장소에서 직접 질의 
- 벡터에 '질문' 을 요청하면서 질문과 유사한 데이터를 검색하도록 한다.
- k 값은 유사한 검색을 했들 때 얻는 데이터 수를 뜻함. 

In [ ]:
print(query)
results = vectorstore_faiss_aws.similarity_search( query, k=1 )   

# 검색된 문장들 출력 (K 값에 따라 문장들의 갯수가 다를수 있음 )
for doc in results:
        print(doc.page_content)

### 6.2 질문을 벡터로 변환하여 검색
- 벡터 저장소에서 질의를 하는데 질문과 유사한 데이터 4개를 찾을때 벡터 값으로 찾는다.
- 따라서 질의 하기전 Amazon Tital 모델에게 질문을 벡터 값으로 변환하도록 한다. 

In [ ]:
print(query)

# Amazon Titan Embede text 모델에게 질문을 벡터로 변환할 것을 요청 
query_vector = embeddings.embed_query(query)

# 변환된 벡터로 벡터 저장소에서 검색하도록 함. 
results = vectorstore_faiss_aws.similarity_search_by_vector(query_vector, k=4)
for doc in results:
    print(doc.page_content)

## 7. RAG(Retrieval-Augmented Generation) 패턴 구현
• 과정:
  1. query로 벡터스토어에서 관련 문서 검색
  2. 검색된 문서를 컨텍스트로 LLM에 전달
  3. LLM이 컨텍스트 기반으로 답변 생성

In [ ]:
chat_llm = ChatBedrock(model_id=base_model, client=bedrock
    )
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print(wrapper_store_faiss.query(query, llm=chat_llm))
